### Некая информация

Доделать:
1. Сойдание папки по имени источника и всех сложенных папок
2. Выбор верхней или нижней площадки (или выбор в зависимости от введенного склонения?)
3. Написать хелп
4. ...

пс: Лена, я тебя прошу, проверяй все вводные параметры на то, что они const в течении всего кода и нигде не переобозначаются


### Модули

In [8]:
import os
import math
import xlwt
import xlrd
import glob
import datetime
import openpyxl
import xlsxwriter

from struct import unpack
from collections import deque
from copy import copy, deepcopy
from openpyxl import load_workbook


import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from numpy import random, mean, var, std, median,arange
from IPython import get_ipython
from matplotlib import mlab

from astropy.utils import iers
from astropy.time import Time, TimeDelta
from astropy import units as u
from bisect import bisect_left
from scipy import signal, misc, linalg as lng

plt.rcParams.update({'figure.max_open_warning': 0})
iers.conf.auto_download = False

### Ввод параметров

In [3]:
"""path_name = str(input('Имя источника: '))
data_process = str(input('Дата обработки: '))

try:
    path = '/home/lena/marina/' + path_name + '/'
    os.mkdir(path)
    
except Exception as e: 
    print('Такая папка уже существует.')
    
    
try:
    path2 = path + 'txt/'
    os.mkdir(path2)   
except Exception as e: 
    print('')

a = float(input('Прямое восхождение alfa (h): '))
b = float(input('Прямое восхождение (m): '))
c = float(input('Прямое восхождение alfa (s): '))

alfa = [a,b,c]

a = float(input('Склонение delta (grad): '))
b = float(input('Склонение delta (m): '))
c = float(input('Склонение delta (s): '))

delta = [a,b,c]

DM = float(input('Мера дисперсии (*10**-3): '))

REGISTER = str(input('Номер регистратора [N1/N2]: '))

DM = DM*10**-3
W50 = float(input('Ширина импульса (мс): '))

module = int(input('Модуль: '))
ray = int(input('Луч: '))
SNR = float(input('От какого С\Ш рабоать?: '))
image2 = str(input('Сохранять картинки 7-ми минутных записей? [y/n]: '))
image = str(input('Сохранять картинки иdfdндивидуальных импульсов? [y/n]: '))"""


#path_name = str(input('Имя источника: '))
path_name = str('SGR1935+2154')


#data_process = str(input('Дата обработки: '))
data_process = str('25.08.2016')


try:
    path = '/home/lena/marina/' + path_name + '/'
    os.mkdir(path)
    
except Exception as e: 
    print('Такая папка уже существует.')
    
    
try:
    path2 = path + 'txt/'
    os.mkdir(path2)   
except Exception as e: 
    print('')

#a = float(input('Прямое восхождение alfa (h): '))
a = 19
#b = float(input('Прямое восхождение (m): '))
b = 34
#c = float(input('Прямое восхождение alfa (s): '))
c = 55.68

alfa = [a,b,c]

#a = float(input('Склонение delta (grad): '))
a = 21
#b = float(input('Склонение delta (m): '))
b = 53
#c = float(input('Склонение delta (s): '))
c = 48.2
delta = [a,b,c]

#DM = float(input('Мера дисперсии (*10**-3): '))
DM = 332.72

#REGISTER = str(input('Номер регистратора [N1/N2]: '))
REGISTER = str('N1')


DM = DM*10**-3

#W50 = float(input('Ширина импульса (мс): '))
W50 = float(50) # это случайное число, в данном случае не пригодится

#module = int(input('Модуль: '))
module = int(6)

ray = int(7)

#ray = int(input('Луч: '))

#SNR = float(input('От какого С\Ш рабоать?: '))
SNR = float(7)

#image2 = str(input('Сохранять картинки 7-ми минутных записей? [y/n]: '))
#image = str(input('Сохранять картинки иdfdндивидуальных импульсов? [y/n]: '))

image2 = str('y')
image = str('y')



"""path_name = 'J0139+3310'
data_process = ''
alfa = [1,39,30]
delta = [33,10,0]
DM = 20
DM = DM*10**-3
W50 = 25
REGISTER = 'N1'
module = 3
ray = 5

SNR = 5
image2 = 'n'
image = 'y'

try:
    path = '/home/lena/marina/' + path_name + '/'
    os.mkdir(path)
    
except Exception as e: 
    print('Такая папка уже существует.')
    
    
try:
    path2 = path + 'txt/'
    os.mkdir(path2)   
except Exception as e: 
    print('')"""




"""path_name = 'J0659+14'
data_process = '01.06.2018 - 31.12.2019'

try:
    path = '/home/lena/marina/' + path_name + '/'
    os.mkdir(path)
    
except Exception as e: 
    print('Такая папка уже существует.')
    
    
try:
    path2 = path + 'txt/'
    os.mkdir(path2)   
except Exception as e: 
    print('')
    
    

alfa = [6,59,48]
delta = [14,14,19]
DM = 13.94
DM = DM*10**-3
W50 = 14.6
REGISTER = 'N2'
module = 2
ray = 6

SNR = 7
image2 = 'n'
image = 'y'"""

'import argparse\ndef parse_args():\n    parser = argparse.ArgumentParser()\n    parser.add_argument("-i", "--iter", dest=\'iter\', nargs=\'+\', required=True, help="Help string")\n    return parser.parse_args()\n\narguments = (parse_args())\nZ = arguments.__dict__\nZ = str(Z)\nif len(Z) == 15:\n    Z = int(Z[11])\nelse: \n    Z = int(Z[11:13])'

### Звездное время и всякие поправки

In [1]:
def star_time(time, delta):
    t1900 = Time('1900-01-01T12:00:00', format='isot', precision=7)
    utd = (time.jd - t1900.jd + 1)
    utd2 = utd - 36525
    utcen = utd2 / 36525
    ts0 = 6 * 3600 + 41 * 60 + 50.54841 + 8640184.812866 * utcen + 0.093104 * utcen * utcen - 6.2e-6 * utcen * utcen * utcen
    td0 = ts0 / 86400
    td0 = td0 + ((time.datetime.hour + time.datetime.minute / 60 +
                  time.datetime.second / 3600) / 24) * 1.002737909
    
    td0 = td0 - int(td0)
    if td0 < 0:
        td0 += 1
    tdh = td0 * 24
    return tdh


def culmination_utc_to_sideral(time):
    a = (str(time)[0:11] + '00:00:00.0000000')
    time2 = Time(a, format='isot', precision=7)
    
    #time2 = Time('2018-08-25T00:00:00.000', format='isot', precision=7)
    s0 = star_time(time2, delta)
    hour = time.datetime.hour + time.datetime.minute/60 + time.datetime.second/3600
    s_hour = hour / (1 - 236.55/(24*3600))#;    // столько звездных часов прошло с начала дня до кульминации
    if (s_hour < 0):
        s_hour = s_hour + 24.
    s_culm = s0 + s_hour
    if ( s_culm > 24.):
        s_culm = s_culm - 24.
    return s_culm

def add_lambda_bsa1(s_culm):
    lambda_grad = (37 + 37 / 60 + 48 / 3600) / 360
    lambda_hour = lambda_grad * 24
    s_culm1 = s_culm + lambda_hour
    if (s_culm1 < 0):
        s_culm1 = s_culm1 + 24
    return s_culm1

def add_incline_bsa1(alfa, delta):
    fi = 0.956829
    be = 0.008436
    rg = 1.7453292e-2
    rm = 2.90888e-4
    rs = 4.8481368e-6
    am = 46.1*rs
    an = 20.0*rs
    alfa = (alfa[0] * rg + alfa[1] * rm + alfa[2] * rs) * 15
    delta = (delta[0] * rg + delta[1] * rm + delta[2] * rs)
    
    aa = (((np.sin(fi))**2) * ((np.cos(be))**2) + ((np.cos(fi))**2))
    bb = 2 * (np.sin(fi)) * (np.cos(be)) * np.sin(delta)
    cc = (np.sin(delta))**2 - (np.cos(fi))**2

    x = (bb + ((bb)**2 - 4. * aa * cc)**0.5) / (2 * aa)
    y = x * np.sin(be) / np.cos(delta)
    z = (1. - (y * y))**0.5

    t = y / z
    dalfa = np.arctan(t)
    alfa = alfa + dalfa

    if alfa < 0:
        alfa = alfa + 2 * np.pi

    RG = (np.pi / 180.)
    RM = (np.pi / 180.) / 60.
    RS = (np.pi / 180.) / 3600.
    hour = int(alfa / (RG * 15))
    mint = int((alfa - (hour * RG * 15.)) / (RM * 15.))
    sec = (alfa - hour * RG * 15. - mint * RM * 15.) / (RS * 15.)
    hour = hour + mint / 60. + sec / 3600;
    while (hour >= 24):
        hour = hour - 24
    return hour

def precess1(coord, time, delta):
    RG = (np.pi / 180.)
    RM = (np.pi / 180.) / 60.
    RS = (np.pi / 180.) / 3600.
    am = 46.1 * RS
    an = 20.4 * RS
    alfa = (int(coord[0]) + int(coord[1]) / 60 + coord[2] / 3600) * 15 * np.pi / 180
    delta = (int(delta[0]) + int(delta[1]) / 60 +
             delta[2] / 3600) * np.pi / 180
    
    t2000 = Time('2000-01-01T12:00:00')
    t = (time.jd - t2000.jd)/365
    alfa1 = alfa
    delta1 = delta

    for i in range(2):
        alf = alfa - (am + an * np.sin(alfa1) * np.tan(delta1)) * t
        del1 = delta - (an * np.cos(alfa1)) * t
        alfa1 = (alf + alfa) / 2
        delta1 = (del1 + delta) / 2
        
    hour = int(alf / (RG * 15))
    mint = int((alf - (hour * RG * 15.)) / (RM * 15))
    sec = (alf - hour * RG * 15 - mint * RM * 15) / (RS * 15)
    
    while (hour >= 24):
        hour = hour - 24
        
    alfa2 = [hour, mint, sec]
    grad0 = del1 * 180. / np.pi
    igrad = int(grad0)
    min0  = (grad0 - igrad)*60
    imin  = int((grad0 - igrad)*60)
    sec = (min0 - imin)*60
    delta = [igrad, imin, sec]

    if hour >= 24:
        hour = hour - 24
    
    alfa2 = hour +  mint/60 +  sec/3600
    return alfa2, delta

def my_sidereal_time(time, delta):
    s_culm = culmination_utc_to_sideral(time)
    s_culm = add_lambda_bsa1(s_culm)
    alfa = [0, 0, 0]
    alfa[0] = int(s_culm)
    alfa[1] = int((s_culm - int(s_culm)) * 60)
    a = (s_culm - int(s_culm)) * 60
    alfa[2] = (a - int(a)) * 60
    s_culm = add_incline_bsa1(alfa, delta)
    mint = int((s_culm - int(s_culm)) * 60)
    a = (s_culm - int(s_culm)) * 60
    sec = (a - int(a)) * 60
    ihour = int(s_culm)
    imin = int(mint)
    day = 0
    while (ihour >= 24):
        ihour = ihour - 24
        day += 1
    coord = [ihour, imin, sec]
    alfa2, delta = precess1(coord, time, delta)
    return alfa2


### Открытие только шапки файлов

In [37]:
def read_head_pntr(filename):
    head = {}
    with open(filename, 'rb') as F:
        for i in range(16):
            line = F.readline()
            a, *b = line.decode("utf-8").strip('\n').split()
            head[a] = b
    return head

def time_begin_and_end(head):
    
    if 'UTC' in head['date_begin']:
        # begin
        day, month, year = head['date_begin'][2].split('.')
        hour, minute, second = head['time_begin'][2].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second
                     )
        time_start = Time(isot_time,
                          format='isot',
    
                          location=('37.63d', '54.8394444444d'),
                          precision=3)
        
        #time_start -= 4*u.hour #ПРОВЕРИТЬ
        
        # end
        day, month, year = head['date_end'][0].split('.')
        hour, minute, second = head['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second
                     )
        time_end = Time(isot_time,
                        format='isot',
       
                        location=('37.63d', '54.8394444444d'),
                        precision=3)
        
        time_end -= 4*u.hour
    else:
        # begin
        day, month, year = head['date_begin'][0].split('.')
        hour, minute, second = head['time_begin'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second
                     )
        time_start = Time(isot_time,
                          format='isot',
       
                          location=('37.63d', '54.8394444444d'),
                          precision=3)
        time_start -= 4*u.hour
        # end
        day, month, year = head['date_end'][0].split('.')
        hour, minute, second = head['time_end'][0].split(':')
        isot_time = (year + '-' + month + '-' + day + 'T' +
                     hour + ':' + minute + ':' + second
                     )
        time_end = Time(isot_time,
                        format='isot',
       
                        location=('37.63d', '54.8394444444d'),
                        precision=3)
        
        time_end -= 4*u.hour

    return time_start, time_end

### Формирование списка файлов

In [40]:
if REGISTER == 'N1':
    reg = 'high-ground'

if REGISTER == 'N2':
    reg = 'low-ground'

if len(data_process) == 0:
    direct = '/bsa/' + reg + '/*/*/*.pnthr'
    F = glob.glob(direct)
    F.sort()

if len(data_process) == 10:
    direct = '/bsa/' + reg + '/' + str(data_process[6:10]) + '/' + str(
        data_process[3:5]) + '*' + '/' + str(data_process[0:2]) + '*.pnthr'

    F = glob.glob(direct)
    F.sort()

if len(data_process) > 10:

    direct1 = '/bsa/' + reg + '/' + str(data_process[6:10]) + '/' + str(
        data_process[3:5]) + '*/' + str(data_process[0:2]) + '*'

    direct2 = '/bsa/' + reg + '/' + str(data_process[19:23]) + '/' + str(
        data_process[16:18]) + '*/' + str(data_process[13:15]) + '*'

    direct3 = '/bsa/' + reg + '/*/*/*.pnthr'
    F = glob.glob(direct1)
    F2 = glob.glob(direct2)
    F3 = glob.glob(direct3)
    F.sort()
    F2.sort()
    F3.sort()
    F3 = F3[F3.index(F[0]): F3.index(F2[-1])]
    F3.sort()
    F = F3

FILE = []
for file in F:
    try:
        head = read_head_pntr(file)
        time_start, time_end = time_begin_and_end(head)
        sidereal_time_start = my_sidereal_time(time_start, delta)
        sidereal_time_end = my_sidereal_time(time_end, delta)
        source = alfa[0] + alfa[1]/60 + alfa[2]/3600
        if source > sidereal_time_start and source < sidereal_time_end:
            FILE.append(file)
            print(file)
    except Exception as e:
        print(e)
        continue


np.savetxt(path + 'Filelist_TEST.txt', FILE, fmt='%s')
FILE = path + 'Filelist_TEST.txt'
F = []
STEP = []
with open(FILE, 'r') as fl:
    for i in range(2500):
        line = fl.readline().strip('\n')
        if len(line) != 0:
            STEP.append(line)

FILE = STEP

### Открытие файлов

In [41]:
def read_pntr(file):
    head = {}
    with open(file, 'rb') as f:
        for i in range(16):
            line = f.readline()
            a, *b = line.decode("utf-8").strip('\n').split()
            head[a] = b

        data = np.fromfile(f, dtype=np.float32)
        data = data.reshape(
            int(head['npoints'][0]), 6, 8, len(head['fbands']) + 1)

    return head, data

### Создание итоговой таблицы

In [ ]:
def append_df_to_excel(filename, df, sheet_name='Sheet', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = load_workbook(filename)
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    if truncate_sheet and sheet_name in writer.book.sheetnames:
        idx = writer.book.sheetnames.index(sheet_name)
        writer.book.remove(writer.book.worksheets[idx])
        writer.book.create_sheet(sheet_name, idx)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
    writer.save()

### Сшивка файлов

In [ ]:
def on_the_border(B, data, file, index1, index2, point_time, mean, REGISTER):
    print('сшитый файл1')
    
    data_32 = [[data[q][module-1][ray-1][x]
                
            for q in range(288120)] for x in (range(32))]
    for i in (range(32)):
        data_32[i] = data_32[i][index1:B]
        
    if REGISTER == 'N1':
        reg = 'high-ground'
    
    if REGISTER == 'N2':
        reg = 'low-ground'

    direct = '/bsa/' + reg + '/*/*/*.pnthr'
    F = glob.glob(direct)
    F.sort()

    A = F.index(file)
    file2 = F[A+1]
    head2, data2 = read_pntr(file2)
    time_start2, time_end2 = time_begin_and_end(head2)

    data_32_2 = [[data2[q][module-1][ray-1][x]
            for q in range(288120)] for x in (range(32))]
    for i in (range(32)):
        data_32_2[i] = data_32_2[i][:int((mean-index1) - (B-mean))]

    data_cut = [ 0 for i in range (32)]
    for i in range(32):
        data_cut[i] = data_32[i] + data_32_2[i]

    point_time = point_time[index1:B] + point_time[:int((mean-index1) - (B-mean))]
    data_32 = data_cut
    return data_32, time_start2, point_time
    
    
    
def on_the_border_2(B, data, file, index1, index2, point_time, mean, REGISTER):
    print('сшитый файл2')
    
    data_32 = [[data[q][module-1][ray-1][x]
            for q in range(288120)] for x in (range(32))]
    for i in (range(32)):
        data_32[i] = data_32[i][:index2]
        
    if REGISTER == 'N1':
        reg = 'high-ground'
    
    if REGISTER == 'N2':
        reg = 'low-ground'

    direct = '/bsa/' + reg + '/*/*/*.pnthr'
    F = glob.glob(direct)
    F.sort()
    A = F.index(file)
    file2 = F[A-1]

    head2, data2 = read_pntr(file2)
    time_start2, time_end2 = time_begin_and_end(head2)

    data_32_2 = [[data2[q][module-1][ray-1][x]
            for q in range(288120)] for x in (range(32))]
    
    for i in (range(32)):
        data_32_2[i] = data_32_2[i][int(B - (index2 - (index2 -mean))):int(B)]


    data_cut = [ 0 for i in range (32)]
    for i in range(32):
        data_cut[i] = data_32_2[i] + data_32[i]
        
    

    point_time = point_time[int(B - (index2 - (index2 -mean))):int(B)] + point_time[:int(index2)]
    data_32 = data_cut

    return data_32, time_start2, point_time

### Обработка

In [1]:
def data_processing(file, alfa, delta, module, ray, DM, W50, image, iteration,
                    path, path_name, SNR, image2):

    from astropy.utils import iers
    from astropy.time import Time, TimeDelta

    import os.path
    chek = 0
    check_file = os.path.exists(file)
    if check_file == False:
        chek += 1
        return chek

    head, data = read_pntr(file)
    time_start, time_end = time_begin_and_end(head)

    def moving_average(a, n=3):
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        b = ret[n - 1:] / n
        return b

    b = str(head['date_begin'][0])
    bb = [b[0:2:], b[3:5:], b[6:10:]]
    bb = list(reversed(bb))
    year = bb[0]
    month = bb[1]
    if int(month[0]) == 0:
        month = month[1::]
    day = bb[2]
    if int(day[0]) == 0:
        day = day[1::]
    t0 = int(year) + int(month) / 12 + int(day) / 365

    source = alfa[0] + alfa[1] / 60 + alfa[2] / 3600

    dt = TimeDelta(0.0124928, format='sec')
    b = 288120
    num = int(9604)

    time = [(time_start + dt * i) for i in (range(0, b, num))]
    sidereal_time = []
    for i in (range(len(time))):
        sidereal_time.append(my_sidereal_time(time[i], delta))

    b = 288120

    def where(time_s, source):
        pos = bisect_left(time_s, source)
        before = time_s[pos - 1]
        after = time_s[pos]
        if after - source < source - before:
            return after
        else:
            return before

    mean1 = where(sidereal_time, source)
    point_time = [i for i in (range(b))]
    mean = point_time[(sidereal_time.index(mean1)) * num]
    time = [(time_start + dt * i)
            for i in (range(mean - int(num / 2), mean + int(num / 2)))]
    sidereal_time = []
    for i in (range(len(time))):
        sidereal_time.append(my_sidereal_time(time[i], delta))
    mean1 = where(sidereal_time, source)
    mean2 = (sidereal_time.index(mean1))
    B = str(head['npoints'])
    B = int(B[2:-2:])
    if mean2 <= (len(sidereal_time) / 2):
        mean3 = point_time[point_time.index(
            mean)] - len(sidereal_time) / 2 + mean2

    else:
        mean3 = point_time[point_time.index(mean)] + abs(
            len(sidereal_time) / 2 - mean2)

    mean = int(mean3)
    index = int(
        round(
            7 * 60 * 1000 / 12.4928 /
            np.cos(math.radians(delta[0] + delta[1] / 60 + delta[2] / 3600)) /
            2))
    index1 = mean - index
    index2 = mean + index
    cos = np.cos(math.radians(delta[0] + delta[1] / 60 + delta[2] / 3600))
    if REGISTER == 'N1':
        reg = 'high-ground'

    if REGISTER == 'N2':
        reg = 'low-ground'

    direct = '/bsa/' + reg + '/*/*/*.pnthr'
    F = glob.glob(direct)
    F.sort()
    CONST1 = 0
    CONST2 = 0

    if (B - mean) < index:

        data_32, time_start2, point_time = on_the_border(
            B, data, file, index1, index2, point_time, mean, REGISTER)
        data_32.reverse()
        data_32_origin = copy(data_32)
        CONST1 += 1

    else:
        if mean < index:
            data_32, time_start2, point_time = on_the_border_2(
                B, data, file, index1, index2, point_time, mean, REGISTER)
            data_32.reverse()
            data_32_origin = copy(data_32)
            CONST2 += 1

        else:
            data_32 = [[
                data[q][module - 1][ray - 1][x] for q in range(288120)
            ] for x in (range(32))]
            data_32.reverse()
            point_time = point_time[index1:index2]
            data_32_origin = copy(data_32)
            for i in (range(32)):
                data_32[i] = data_32[i][index1:index2]

    if len(data_32) == 0:
        return chek

    b = 288120
    data_32_wdm = copy(data_32)
    n, t, t2 = [], [], []
    for i in range(32):
        n.append((float((head['fbands'][i])[0:-1:]))**-2)

    for i in range(32):
        t.append(((4.148808 * (10**6) * (n[len(n) - 1] - n[i]) * DM)))

    for i in range(32):
        if t[i] == 0:
            t2.append(0)
        else:
            t2.append((round(t[i] / (0.0124928))))

    for i in range(32):
        data_32[i] = data_32[i][t2[i]:] + data_32[i][:t2[i]]

    name = str(file[-21:-12])
    name2 = name
    n2 = 0
    if int(name[-2::][0]) == 0:
        n = int(name[-2::][1])
    else:
        n = int(name[-2::])
        n2 = int(name[-2::])
    n1 = []
    s = [1, 5, 9, 13, 17, 21]
    if n < 22:
        if n2 != 0:
            pos = bisect_left(s, n)
            before = s[pos - 1]
            after = s[pos]
            if after - n < n - before:
                n = str(after)
                if len(str(n)) == 1:
                    n = '0' + str(n)
                a = name[:-2:] + n
            else:
                n = str(before)
                if len(str(n)) == 1:
                    n = '0' + str(n)
                a = name[:-2:] + n
        else:

            pos = bisect_left(s, n2)
            before = s[pos - 1]
            after = s[pos]
            if after - n2 < n2 - before:
                n = str(after)
                if len(str(n)) == 1:
                    N = '0' + str(n)
                a = name[:-2:] + n
            else:
                n = str(before)
                if len(str(n)) == 1:
                    n = '0' + str(n)
                a = name[:-2:] + n
    else:
        a = name[:-2:] + '01'
    name = a

    if n2 != 0:
        file_step = str(file[:-21:] + str(name) + file[-12::])
    else:
        file_step = str(file[:-21:] + str(name) + file[-12::])

    head_step, data_step = read_pntr(file_step)

    data_step_32 = [0 for i in range(32)]
    for i in range(32):
        data_step_32[i] = [j[module - 1][ray - 1][i] for j in data_step]
    ACP = []
    for i in range(32):
        ACP.append(max(data_step_32[0]) / max(data_step_32[i]))

    q = len(data_32)
    q2 = len(data_32[0])
    for i in range(q):
        for k in range(q2):
            data_32[i][k] = data_32[i][k] / ACP[i]
            data_32_wdm[i][k] = data_32_wdm[i][k] / ACP[i]

    summ_ray2 = [0 for i in range(len(data_32[0]))]
    summ_wdm_ray2 = [0 for i in range(len(data_32[0]))]
    for i in range(q):
        for k in range(q2):
            summ_ray2[k] += data_32[i][k]
            summ_wdm_ray2[k] += data_32_wdm[i][k]

    summ_ray2 = np.array(summ_ray2)
    summ_wdm_ray2 = np.array(summ_wdm_ray2)

    def flatter (summ_ray2, summ_wdm_ray2):
        M = []
        k = -1
        for i in (range(len(summ_ray2))):
            try:
                n = 100
                if i % n == 0:
                    k += 1
                    null_cut = summ_ray2[int(k * n):int(k * n + n)]
                    null_cut_wdm = summ_wdm_ray2[int(k * n):int(k * n + n)]
                    M.append(np.median(null_cut_wdm)) 
            except Exception as e:
                print(e)
                continue

        median = []
        for i in range(len(M)-1):
            a = np.linspace(M[i],M[i+1],n)

            for k in range(len(a)):
                median.append(a[k])

        summ_ray22 = summ_ray2[:len(median)]
        b =  summ_ray22 - median

        """fig, ax = plt.subplots(figsize=(8, 6))
        plt.plot(summ_ray22)
        plt.plot(median)

        fig, ax = plt.subplots(figsize=(8, 6))
        plt.plot(b)"""

        return b
    


    summ = flatter(summ_ray2[500:], summ_wdm_ray2[500:])
    summ_wdm = flatter(summ_wdm_ray2[500:] , summ_wdm_ray2[500:])

    W = W50 / 12.4928
    std, acp, point, std_high, time_utc, time_jd2000 = [], [], [], [], [], []
    k = -1
    d1 = 0
    # PolyOrder = #ЗАПОЛНИ
    def averaging(a, step_averaging):
        b = []
        c = int(len(a)//step_averaging)*step_averaging
        a = a[:c]
        a = np.reshape(a, (int(len(a)//step_averaging), step_averaging))
        for i in range(len(a)):
            b.append(sum(a[i]))
        a = b
        return a

    def moving_average(a, step_moving_average):
            n = step_moving_average
            ret = np.cumsum(a, dtype=float)
            ret[n:] = ret[n:] - ret[:-n]
            b = ret[n - 1:] / n
            return b

    def infnorm(vector):
        norm = 0
        for i in range(len(vector)):
            if abs(vector[i]) > norm:
                norm = vector[i]
        return norm

    def normalize(vector):
        norm = infnorm(vector)
        return [v / norm for v in vector]

    null2 = list(summ)
    null_wdm = list(summ_wdm)
    null_wdm2 = list(summ_wdm)
    null_wdm22 = list(summ_wdm)


    #step_moving_average = [2, 4, 8, 16, 32,64,128]
    step_moving_average = [2, 4, 8, 16, 32,64,128]
    NULL, TIME, STEP, SNR, POINT = [], [], [], [], []
    null_std  = np.std(null2[:int(len(null2)*0.7)])
    print(null_std)
    g = 5
    null3,point3 = [],[]
    for i in (range(len(null2))):
        if (null2[i] >= g*null_std) and (abs(null2[i]/null_wdm[i]) >= 1.5) and (null2[i]<null_wdm[i]):
            null3.append(null2[i])
            point3.append(point_time[i])

    print(len(point3), point3)
    print(point_time[0])

    
    for i in (range(len(null3))):
        try:
            k = null2.index(null3[i])
            for z in range(len(step_moving_average)):
                null_average = moving_average(null2, step_moving_average[z])
                TIME.append([])
                NULL.append([])
                STEP.append([])
                SNR.append([])
                POINT.append([])

                if CONST1 != 0:
                    if point_time[k] <= B and point_time[k] > index:
                            T = time_start + dt * point_time[k]
                    else:
                        T = time_start2 + dt * point_time[k]

                if CONST2 != 0:
                    if point_time[k] <= index2:
                        T = time_start + dt * point_time[k]
                    else:
                        T = time_start2 + dt * point_time[k]

                if CONST1 == 0 and CONST2 == 0:

                    T = time_start + dt * point_time[k]
                    TIME[z].append(T)
                    NULL[z].append(null_average[k])
                    STEP[z].append(step_moving_average[z])
                    SNR[z].append(null_average[k]/std)
                    POINT[z].append(point_time[k])

        except Exception as e:
            print(e)
            continue
        
    point_list = []    
    print(len(POINT[1]))
    for i in range(len(POINT)):
        for k in range(len(POINT[i])):
            point_list.append(POINT[i][k])

    point_list = list(set(point_list))
    point_list.sort()
    print(POINT)

    snr_max, step_max, point_max, null_max = [],[],[], []
    for i in range(len(point_list)):
        snr_point, step_point, null_point = [], [], []
        for k in range(len(POINT)):
            if point_list[i] in POINT[k]:
                snr_point.append(SNR[k][POINT[k].index(point_list[i])])
                step_point.append(STEP[k][POINT[k].index(point_list[i])])
                null_point.append(NULL[k][POINT[k].index(point_list[i])])


            else:
                snr_point.append(0)
                step_point.append(0)
                null_point.append(0)
        snr_max.append(max(snr_point))
        step_max.append(step_point[snr_point.index(max(snr_point))])
        point_max.append(point_list[i])
        null_max.append(max(null_point))

    print('кол-во импульсов (2)', len(point_max))
    
    for i in (range(len(point_max))):
        try:
            dlt = 2
            m = point_max[i]
            #if CONST1 != 0 or CONST2 !=0:
            #m = point_time.index(point_max[i])
            """index = round(1000 / 12.4928)
            if abs(t2[0] - t2[-1]) > (1000 / 12.4928):
                index = round(2500 / 12.4928)"""

            index = round(N/ 2)
            index1 = m - index
            index2 = m + index
            y_data, y_new, y_new2, m_a = [], [], [],[]
            for z in range(32):
                y_data.append(data_32_origin_ray2[z][index1:index2])
            for k in range(32):
                a = normalize(y_data[k])
                b = moving_average(a, step_max[i])
                y_new2.append(b)
                y_new.append(a)

            m = point_time.index(point_max[i])
            i1 = m - index

            i2 = m + index
            a = point_time[i1:i2]
            print(i1,i2)
            print(len(a))
            b1 = null_wdm[i1:i2]
            #b12 = null_wdm2[i1:i2]  # вырезаем без дисперсии
            b = null2[i1:i2]  # с дисперсией
            #b2 = null2[i1:i2]

            #Apr = flatter(b, b1)
            #Apr = np.hstack(Apr)
            #bb = Apr
            bb = b

            #Apr = flatter(b1, b1)
            #Apr = np.hstack(Apr)
            #bb1 = Apr
            bb1 = b1

            a = [l * 12.4928 for l in range(len(b1))]
            print(step_max[i])
            sum_step = averaging(bb, step_max[i])
            sum_step_x = [l * 12.4928 for l in range(len(sum_step))]

            fig, ax = plt.subplots(num=None,
                                   figsize=(12, 12),
                                   facecolor='w',
                                   edgecolor='k')

            plt.subplot(2, 2, 1)
            #plt.title('S/N = ' + str(round(std_high[i], 2)), fontsize=14)
            plt.xlabel('Время [мс]', fontsize=12)
            sns.axes_style('whitegrid')
            plt.ylabel("Плотность потока [АЦП]", fontsize=12)
            heatmap_plot = sns.heatmap(y_new,
                                       cmap="Greys",
                                       cbar=False,
                                       xticklabels=False,
                                       yticklabels=n)

            plt.subplot(2, 2, 2)
            plt.xlabel('Время [мс]', fontsize=12)
            a = a[:len(bb)]
            plt.ylabel("Плотность потока [АЦП]", fontsize=12)
            plt.plot(a, bb, color='red', label='полсе компенсации дисперсии')
            plt.plot(a, bb1, color='blue', label='до компенсации дисперсии')
            #plt.plot(a, b1, color='red', label='полсе компенсации дисперсии')
            #plt.plot(a, b, color='red', label='полсе компенсации дисперсии')
            #plt.plot(a, Apr, color='green', label='полсе компенсации дисперсии')

            plt.subplot(2, 2, 3)
            #plt.title('S/N = ' + str(round(std_high[i], 2)), fontsize=14)
            plt.xlabel('Время [мс]', fontsize=12)
            sns.axes_style('whitegrid')
            plt.ylabel("Плотность потока [АЦП]", fontsize=12)
            heatmap_plot = sns.heatmap(y_new2,
                                       cmap="Greys",
                                       cbar=False,
                                       xticklabels=False,
                                       yticklabels=n)

            plt.subplot(2, 2, 4)
            plt.plot(sum_step_x, sum_step, color='red', label='полсе компенсации дисперсии')




            plt.savefig(path +  '/' + str(point_max[i]) + '__' + str(name2) + '_step: ' + str(step_max[i]) + '.png',
                        dpi=None,
                        facecolor='w',
                        edgecolor='w',
                        orientation='portrait');
            #plt.close();
        except Exception as e:
            print(e)
            continue

    print('Обработал файл ', str(file))


    return chek

IndentationError: expected an indented block (<ipython-input-1-97fd29a72a11>, line 11)

### Запуск обработки и сохранение результатов 

In [2]:
wb = openpyxl.Workbook()
wb.save(path + path_name + '.xlsx')
finish_file = []
iteration = 0
chek = 0
for file in FILE:
    try:
        finish_file.append(file)
        np.savetxt(path + 'finish_file.txt', finish_file, fmt='%s')
        chek = 0
        chek = data_processing(file, alfa, delta, module, ray, DM, W50, image, iteration, path, path_name, SNR, image2)
        iteration += 1       
        if chek != 0:
            import telebot
            TEXT = 'Я упала :С'
            bot = telebot.TeleBot('758732793:AAFENzzr9SzaEyU1TjodolZ0_8g32gvvQZc')
            bot.config['api_key'] = '758732793:AAFENzzr9SzaEyU1TjodolZ0_8g32gvvQZc'
            bot.get_me()
            bot.send_message(chat_id = 383266287, text = str(TEXT))
            break
    except Exception as e:
        if chek != 0:
            import telebot
            TEXT = 'Я упала :С'
            bot = telebot.TeleBot('758732793:AAFENzzr9SzaEyU1TjodolZ0_8g32gvvQZc')
            bot.config['api_key'] = '758732793:AAFENzzr9SzaEyU1TjodolZ0_8g32gvvQZc'
            bot.get_me()
            bot.send_message(chat_id = 383266287, text = str(TEXT))
            break
        
        print(e)
        continue

NameError: name 'openpyxl' is not defined

In [3]:
round(25/12.4928)*3

6